First we start by using the `%maven` magic to add the latest Calcite version as a dependency.

In [1]:
%maven org.apache.calcite:calcite-core:1.19.0

We now construct a `SqlParser` instance and parse a simple query.

In [2]:
import org.apache.calcite.sql.SqlNode;
import org.apache.calcite.sql.parser.SqlParser;

SqlParser parser = SqlParser.create("SELECT * FROM \"foo\" WHERE \"bar\"=3 GROUP BY \"baz\"");
SqlNode query = parser.parseQuery();

The result gives a `SqlNode` that represents the parsed query. There are a number of useful things we can do with this. Firstly, we can convert it back into a string. Note that this string will by default use the ANSI SQL dialect and apply some pretty printing, so it won't be the exact SQL we started out with.

In [3]:
query.toString()

SELECT *
FROM `foo`
WHERE `bar` = 3
GROUP BY `baz`

From here, there are two main approaches we can take to getting what we want out of the query.
Since we know this is a `SELECT` statement, we can cast the `SqlNode` to `SqlSelect` and pull what we need from there.
You can see more of the available methods [in the Javadoc](https://calcite.apache.org/apidocs/org/apache/calcite/sql/SqlSelect.html).

In [4]:
import org.apache.calcite.sql.SqlSelect;

SqlSelect select = (SqlSelect) query;

System.out.println(select.getSelectList());
System.out.println(select.getWhere());
System.out.println(select.getGroup());

*
`bar` = 3
`baz`


Another approach is to implement the `SqlVisitor` interface.
The visitor below simply prints all literals and identifiers it encounters.

In [5]:
import org.apache.calcite.sql.SqlIdentifier;
import org.apache.calcite.sql.SqlLiteral;
import org.apache.calcite.sql.util.SqlBasicVisitor;

class PrintingVisitor extends SqlBasicVisitor<Void> {
  public Void visit(SqlLiteral literal) {
    System.out.println("Literal: " + literal);
    return null;
  }
  
  public Void visit(SqlIdentifier id) {
    System.out.println("Identifier: " + id);
    return null;
  }
}

query.accept(new PrintingVisitor());

Identifier: *
Identifier: foo
Identifier: bar
Literal: 3
Identifier: baz
